## Getting Started with Prompt Management and Flows for Amazon Bedrock

> Notebook [original](https://github.com/aws-samples/amazon-bedrock-samples/blob/main/agents-and-function-calling/bedrock-flows/Getting_started_with_Prompt_Management_Flows.ipynb)

This example shows you how to get started with Prompt Management and Prompt Flows in Amazon Bedrock.

[Amazon Bedrock Prompt Management](https://docs.aws.amazon.com/bedrock/latest/userguide/prompt-management.html) streamlines the creation, evaluation, deployment, and sharing of prompts in the Amazon Bedrock console and via APIs in the SDK. This feature helps developers and business users obtain the best responses from foundation models for their specific use cases.

[Amazon Bedrock Prompt Flows](https://docs.aws.amazon.com/bedrock/latest/userguide/flows.html) allows you to easily link multiple foundation models (FMs), prompts, and other AWS services, reducing development time and effort. It introduces a visual builder in the Amazon Bedrock console and a new set of APIs in the SDK, that simplifies the creation of complex generative AI workflows.

Let's start by making sure we have the lastest version of the Amazon Bedrock SDK, importing the libraries, and setting-up the client.

In [13]:
import boto3
import json
from datetime import datetime

Note the Prompt Management and Flows features are part of the Bedrock Agent SDK.

In [14]:
bedrock_agent = boto3.client(service_name = "bedrock-agent", region_name = "us-west-2")

## Prompt Management

### Create and Manage Prompts

Let's create a sample prompt, in this case for a simple translation task.

In [15]:
agent_instruction = """
You are a friendly virtual assistant in a Primary school. 
Your job is to help students with their queries about school-related topics, such as History o Maths.

Answer the following user query in a friendly and direct way: {{student_user_query}}
"""

In [ ]:
# Define the model ID for the prompt
model_id = "anthropic.claude-3-5-haiku-20241022-v1:0"
model_variant = "haiku-3-5-variant-001"
model_default_variant = "haiku-3-5-variant-001"

# Create a prompt for the customer service use case
# This prompt will be used to generate responses for customer service queries
response = bedrock_agent.create_prompt(
    name = "ecomm-virtual-assistant-claude-3-5-haiku-v001",
    description = "Prompt for out customer service virtual assistant",
    variants = [
        {
            "inferenceConfiguration": {
                "text": {
                    "maxTokens": 4096,
                    "temperature": 0.3,
                    "topP": 0.5
                }
            },
            "modelId": model_id,
            "name": model_variant,
            "templateConfiguration": {
                "text": {
                    "inputVariables": [
                        {
                            "name": "student_user_query"
                        }

                    ],
                    "text": agent_instruction
                }
            },
            "templateType": "TEXT"
        }
    ],
    defaultVariant = model_default_variant
)

print(json.dumps(response, indent=2, default=str))
promptId = response["id"]
promptArn = response["arn"]
promptName = response["name"]
print(f"Prompt ID: {promptId}\nPrompt ARN: {promptArn}\nPrompt Name: {promptName}")

Now that we have a draft prompt, we can create versions from it.

In [ ]:
response = bedrock_agent.create_prompt_version(
    promptIdentifier = promptId
)
print(json.dumps(response, indent=2, default=str))

Here we can see the list of prompts in our Prompt Library or catalog.

Example: delete a prompt
```
response = bedrock_agent.delete_prompt(
    promptIdentifier='4TX3FL6O17'
)
```

In [ ]:
response = bedrock_agent.list_prompts(
    maxResults = 10
)
print(json.dumps(response["promptSummaries"], indent=2, default=str))

We can also read the details of any of our prompts.

In [ ]:
response = bedrock_agent.get_prompt(
    promptIdentifier = promptId,
    promptVersion = "1"
)
print(json.dumps(response, indent=2, default=str))

-------

### Ejemplo: Todo en uno

In [ ]:
def filter_by_name(response_data, name_to_find):
    # Get the list of prompts from promptSummaries
    prompts = response_data.get('promptSummaries', [])
    
    # Filter by name
    return [item for item in prompts if item.get('name') == name_to_find]

In [ ]:
# SDK initialization
bedrock_runtime = boto3.client('bedrock-runtime', region_name='us-west-2')

def call_bedrock_with_prompt(user_prompt, prompt_name="ecomm-virtual-assistant-claude-3-5-haiku-v001"):
    # List available prompts
    all_prompts = bedrock_agent.list_prompts()

    # Get our prompt
    filtered_prompt = filter_by_name(all_prompts, prompt_name)
    filtered_prompt_arn = filtered_prompt[0]["arn"]

    # Call the model using the prompt
    response = bedrock_runtime.converse(
      modelId=filtered_prompt_arn,
      promptVariables={
        'ecomm_user_query': {
            'text': user_prompt
        }
      }   
    )

    return response

In [ ]:
# User sample query
user_prompt="Hi! Do you have video games?"
response = call_bedrock_with_prompt(user_prompt)

In [ ]:
# Print the response
response_text = response['output']['message']['content'][0]['text']
print(response_text)